This notebook demonstrates the use of [GDPTools](https://gdptools.readthedocs.io/en/develop/index.html) for spatially interpolating (Aggregation) Conus404 data to a set of HUC12s in the Delaware River Basin.

## Summary
This use-case demonstrates a common spatial interpolation pattern in hydrologic sciences.  Interpolating gridded data, in this case the Conus404 dataset, to Polygons representing basins, such as HUC12s.  GDPTools is a pyhton package for Geospatial Interpolation, developed by the USGS NHGF project.  We use GDPTools to generate a weights file for area-weighted spatial interpolation, interpolating the time-dependend gridded data to the HUC12s. The [Conus404](https://www.sciencebase.gov/catalog/item/6372cd09d34ed907bf6c6ab1) data release describes Conus404 as "... a unique, high-resolution hydro-climate dataset appropriate for forcing hydrological models and conducting meteorological analysis over the contiguous United States".  This use-case uses the HyTest intake catalog which provides a convenient method to access the Conus404 data.  

## Open Source python Tools
- gdptools: GeoDataProcessing Tools (GDPTools) is a python package for geospatial interpolation
- [HyTest intake catalog](https://github.com/hytest-org/hytest/tree/main/dataset_catalog)
- [NHGF data service]( https://labs.waterdata.ugsgs.gov) clients:
  - We use the collection of [HyRiver](https://docs.hyriver.io/examples/notebooks/nhdplus.html) Packages for accessing a set of HUC12 polygons in the Delaware River Basin programatically using the [NLDI](https://docs.hyriver.io/readme/pynhd.html) and [WaterData](https://docs.hyriver.io/readme/pynhd.html)

## Imports


In [3]:
# Common python packages
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import hvplot.dask
import intake
import warnings
import intake_xarray
import datetime
import holoviews as hv
import numpy as np
import pandas as pd
import geopandas as gpd

# HyRiver packages
from pynhd import NLDI, WaterData

# GDPTools packages
from gdptools import AggGen, UserCatData, WeightGen


hv.extension("bokeh")
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'xarray'

## GDPTools version
Note this use-case is currently using the following version of GDPTools.  GDPTools is in an active state of development, and if the viewer of this use-case would like to use GDPTools for their work, it would be worth checking if you have the most up-to-date version.


In [1]:
!conda list gdptools

/bin/sh: conda: command not found


## HyRiver Packages
Here we use two HyRiver clients, `NLDI` and `WaterData1` to:
- Define the upstream basin at a USGS gage towards the downstream reach of the Delaware River
- Use the basin geometry to extract HUC21s within the geometry.


In [2]:
# USGS gage 01482100 Delaware River at Del Mem Bridge at Wilmington De
gage_id = '01482100'
nldi = NLDI()
del_basins = nldi.get_basins(gage_id)
huc12_basins = WaterData('huc12').bygeom(del_basins.geometry[0])

NameError: name 'NLDI' is not defined

### View the content of the resulting GeoDataFrame


In [ ]:
huc12_basins.head()

### Note the crs associate with the GeoDataFrame
This value will be used later.


In [ ]:
huc12_basins.crs

### Reduce the context of the GeoDataFrame to only the columns necessary for the analysis
GDPTools requires only the column used to identify the basins and the geometry.  To reduce the memory overhead of the calculations below, it's a good practice to remove unneccessary columns.  First we list the columns and then we keep only `huc12` and `geometry`.  


In [ ]:
print(huc12_basins.columns)
working_gdf = huc12_basins[["huc12", "geometry"]]

# Plot the resulting GeoDataFrame


In [ ]:
from holoviews.element.tiles import OSM
drb = working_gdf.hvplot(
    geo=True, coastline='50m', alpha=0.2,  frame_width=300,
    xlabel="longitude", ylabel="latitude", clabel="monthly water evaporation (mm)",
    title="Delaware River HUC12 basins", xlim=(-76.8, -73.8), aspect='equal'
)
OSM() * drb

## HyTest Intake Catalog and Conus404
Further examples of using the HyTest catalog can be found [here](https://github.com/hytest-org/hytest/tree/main/dataset_catalog).  In the following cells we step into the catalog to retrieve the `conus404-daily-osn`.  The data with "-osn" extensions can be accessed without authentication.

### Open the catalog


In [ ]:
# open the hytest data intake catalog
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)

### Open the conus404 catalog


In [ ]:
# open the conus404 sub-catalog
cat = hytest_cat['conus404-catalog']
list(cat)

### Open the daily data


In [ ]:
## Select the dataset you want to read into your notebook and preview its metadata
dataset = 'conus404-daily-osn' 
cat[dataset]

## Load conus404 daily


In [ ]:
print(f"Reading {dataset} metadata...", end='')
ds = cat[dataset].to_dask().metpy.parse_cf()
print("done")

### View the xarray representation


In [ ]:
ds

### Conus404 projection


In [ ]:
c404_proj = "+proj=lcc +lat_0=39.1000061035156 +lon_0=-97.9000015258789 +lat_1=30 +lat_2=50 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs"

## GDPTools
In the following cells GDPtools if first used to generate a weighting file used for area-weighted statistics and secondly to apply the weighting file to interpolate the Conus404 gridded data to each of the HUC12 polygons using an area-weighted mean.  GDPTools has a number of data classes to parameterize the input data, which can be though of as the source and target data with some accompanying attributes of each.  In this case we use the `UserCatData` class.

Source Data:
- ds: Xarray dataset
- x_coord: Name of x-coordinate
- y_cord: Name of y-coordinate
- t_coord: Name of t-coordinate
- proj_ds: The source data's projection.  This can be anything that can by read by [pyproj.crs.CRS.from_user_input()](https://pyproj4.github.io/pyproj/stable/api/crs/crs.html#pyproj.crs.CRS.from_user_input) method.
- var: List of variables in the xarray dataset that will be processed.

Target Data:
- f_feature: The GeoDataFrame - In this case `working_gdf` define above.
- proj_feature: Similar to proj_ds above but for the Target GeoDataFrame.
- id_feature: The column heading of the feature used to identify the target for the interpolation - In this case "huc12"

Period:
The period of interest.  This is where things can go sideways.  It's always a good strategy to start with a small slice of data and work up.  For large Target datasets, for example HUC12s for all of CONUS, you would quickly run out of memory if processing more that a year at a time.  Also some servers have a limit on the size of the data requested.  In this case we have a small example - so it's no issue.


In [ ]:
data_crs = c404_proj
x_coord = "x"
y_coord = "y"
t_coord = "time"
sdate = "1979-10-01T00:00:00.000000000"
edate = "1979-10-07T00:00:00.000000000"
var = ["T2", "ACRAINLSM"]
shp_crs = 4326
shp_poly_idx = "huc12"
wght_gen_crs = 6931

user_data = UserCatData(
    ds=ds,
    proj_ds=data_crs,
    x_coord=x_coord,
    y_coord=y_coord,
    t_coord=t_coord,
    var=var,
    f_feature=working_gdf,
    proj_feature=shp_crs,
    id_feature=shp_poly_idx,
    period=[sdate, edate],
)

### Generate the weights
Generate the weights and save to a file for later use.


In [ ]:
genwghts = True
wght_gen = WeightGen(
    user_data=user_data,
    method="serial",
    output_file="wghts_drb_ser_c404daily.csv",
    weight_gen_crs=6931
)

if genwghts:
    wdf = wght_gen.calculate_weights()

### Aggregate the data using the weights generated above.


In [ ]:
agg_gen = AggGen(
    user_data=user_data,
    stat_method="masked_mean",
    agg_engine="serial",
    agg_writer="netcdf",
    weights='wghts_drb_ser_c404daily.csv',
    out_path='.',
    file_prefix="testing_p"
)
ngdf, ds_out = agg_gen.calculate_agg()

## Generate a plot of the resulting interpolatoin


In [ ]:
aggdata = agg_gen.agg_data.get('T2').da
aggdata

In [ ]:
wvar = 'T2'
time_step = "1979-10-03T00:00:00.000000000"

In [ ]:
vmin = np.nanmin(aggdata.sel(time=time_step).values)
vmax = np.nanmax(aggdata.sel(time=time_step).values)
print(vmin, vmax)

In [ ]:
ngdf['T2'] = ds_out[wvar].sel(time=time_step)
c404_agg = ngdf.to_crs(4326).hvplot(
    c='T2', geo=True, coastline='50m', frame_width=300, alpha=1, cmap='viridis', clim=(vmin, vmax),
    xlabel="longitude", ylabel="latitude", clabel="monthly water evaporation (mm)", xlim=(-76.8, -73.8),
    title="DRB HUC12 area-weighted average", aspect='equal'
)
c404_raw = aggdata.sel(time=time_step).hvplot.quadmesh(
    'lon', 'lat', 'T2', cmap='viridis', alpha=1, grid=True, geo=True, coastline='50m', frame_width=300, clim=(vmin,vmax),
    xlabel="longitude", ylabel="latitude", clabel="monthly water evaporation (mm)", xlim=(-76.8, -73.8),
    title="TerraClimate Monthly AET", aspect='equal'
)

In [ ]:
c404_raw + c404_agg